In [5]:
!pip install transformers datasets torch torchvision


In [2]:
!pip install tf-keras

In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
###fine tune and push the model to hugging face hub
from transformers import ViTForImageClassification, TrainingArguments, Trainer, ViTImageProcessor
from datasets import load_dataset
import torch
from PIL import Image

# Load your dataset
dataset = load_dataset("sagar27kumar/ECG-XRAY-dataset")

# Load the processor and model
processor = ViTImageProcessor.from_pretrained("google/vit-large-patch32-384")
model = ViTForImageClassification.from_pretrained(
    "google/vit-large-patch32-384",
    num_labels=4,  # Number of classes in your dataset
    id2label={0: "Abnormal Heartbeat", 1: "History of MI", 2: "Myocardial Infarction", 3: "Normal Person"},
    label2id={"Abnormal Heartbeat": 0, "History of MI": 1, "Myocardial Infarction": 2, "Normal Person": 3},
    ignore_mismatched_sizes=True
)







# Preprocess the dataset
def preprocess_function(examples):
    images = [Image.open(img_path).convert("RGB") for img_path in examples["image"]]
    inputs = processor(images=images, return_tensors="pt")
    inputs["labels"] = torch.tensor(examples["label"], dtype=torch.long)
    return inputs

# Apply preprocessing to dataset
prepared_dataset = dataset.with_transform(preprocess_function)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=True,
    hub_model_id="sagar27kumar/sagarsahu_ECG-XRAY-ViT",
    no_cuda=True  # Force using CPU
)



# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=prepared_dataset["train"],
    eval_dataset=prepared_dataset["validation"],
    tokenizer=processor,
    data_collator=lambda data: {
        "pixel_values": torch.stack([f["pixel_values"] for f in data]),
        "labels": torch.tensor([f["labels"] for f in data], dtype=torch.long),  # Ensure correct shape and type
    }
)

# Train the model
trainer.train()

# Save and push the model to Hugging Face Hub
trainer.push_to_hub()


In [16]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Define the metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Update the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=prepared_dataset["train"],
    eval_dataset=prepared_dataset["validation"],
    tokenizer=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,  # Add this line
)

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")


/var/folders/l_/sj267b1x1f30mq27_7k16jzh0000gn/T/ipykernel_2083/3324538771.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Evaluation Results: {'eval_loss': 0.09193223714828491, 'eval_model_preparation_time': 0.0022, 'eval_accuracy': 0.9661016949152542, 'eval_precision': 1.0, 'eval_recall': 0.9661016949152542, 'eval_f1': 0.9824482109227872, 'eval_runtime': 132.3123, 'eval_samples_per_second': 3.567, 'eval_steps_per_second': 0.446}


/Users/sagarkumarsahu/Desktop/Heart-disease-prediction-using-ECG/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
